## Задача 5-1. A\* поиск в задаче о кратчайших путях.

В этой задаче Вам предлагается реализовать поиск кратчайших путей в графе с помощью A\*-поиска с использованием эвристической функции («потенциала»), основанном на landmarks. Теоретические основы можно посмотреть [здесь](http://logic.pdmi.ras.ru/midas/sites/default/files/midas-werneck.pdf), слайды 20—36.

Вам предлагается скачать [отсюда](http://www.diag.uniroma1.it/challenge9/download.shtml) файлы “Travel time graph” и “Coordinates” для штата Флорида. Для Вашего удобства они также размещены в архиве `florida.7z` в настоящем репозитории на GitHub.

Функции `read_node_coords` и `read_arcs` возвращают соответственно координаты вершин графа (отнормированные; координаты нужны только для обеспечения возможности выбора landmarks “по периметру графа”) и структура дуг графа.

In [1]:
import math
from typing import List, Dict, Tuple
from collections import namedtuple, defaultdict
Coords = namedtuple('Coords', ['x', 'y'])

def read_node_coords(filename='USA-road-d.FLA.co') -> List[int]:
    node_coords = []
    
    with open(filename, 'r') as coord_file:
        for line in coord_file:
            if line.startswith('v '):
                node_number, x, y = map(int, line.split()[1:])
                node_coords.append(Coords(x, y))
    
    minx = min(c.x for c in node_coords)
    miny = min(c.y for c in node_coords)
    for i, c in enumerate(node_coords):
        node_coords[i] = Coords(c.x-minx, c.y-miny)
    
    return node_coords


def read_arcs(filename='USA-road-t.FLA.gr') -> Dict[int, Dict[int, float]]:
    adjacency_lists = defaultdict(dict)
    
    with open(filename, 'r') as coord_file:
        for line in coord_file:
            if line.startswith('a '):
                node_from, node_to, weight = map(int, line.split()[1:])
                adjacency_lists[node_from-1][node_to-1] = weight
                
    return adjacency_lists

Реализуйте процедуру `good_old_dijkstra`, которая для пары номеров вершин графа ищет кратчайший путь между ними и возвращает список номеров вершин, образующих оптимальный путь и его длину.

In [2]:
import queue

def good_old_dijkstra(adjacency_lists, node_from, node_to=None):
    q = queue.PriorityQueue()
    p = {v: -1 for v in adjacency_lists}
    d = {v: float('inf') for v in adjacency_lists}
    d[node_from] = 0
    q.put((0, node_from))
    while not q.empty():
        dist, u = q.get()
        for v in adjacency_lists[u]:
            edge_len = adjacency_lists[u][v]
            if dist + edge_len < d[v]:
                d[v] = dist + edge_len
                p[v] = u
                q.put((dist + edge_len, v))
    if node_to is None:
        return (p, d)
    start = node_to
    path = []
    while start != -1:
        path.append(start)
        start = p[start]
    return (reversed(path), d[node_to])

Реализуйте тройку процедур `choose_landmarks`, `precalculate_landmark_distances` и `a_star_with_landmarks`. Процедура `choose_landmarks` выбирает нужное количество специальных вершин графа — этот выбор делается равномерным выбором по периметру графа (см. слайд 30 в [презентации](http://logic.pdmi.ras.ru/midas/sites/default/files/midas-werneck.pdf)). Процедура `precalculate_landmark_distances` для каждой вершины из заданного набора с помощью обычного алгоритма Дейкстры вычисляет расстояния до всех вершин графа. Эта информация затем используется в `a_star_with_landmarks` для ускорения поиска кратчайшего пути.

In [9]:
import numpy as np
def choose_landmarks(node_coords, num_landmarks=15):
    xs = np.array([v.x for v in node_coords])
    ys = np.array([v.y for v in node_coords])
    
    mean_x = np.mean(xs)
    mean_y = np.mean(ys)
    mean = np.array([mean_x, mean_y])
    sectors = [[] for i in range(num_landmarks)]
    for i, v in enumerate(node_coords):
        alpha = math.atan2(v.x - mean_x, v.y - mean_y)
        while alpha < 0:
            alpha += 2 * math.pi
        sect_coord = int(alpha * num_landmarks / (2 * math.pi))
        sectors[sect_coord].append((np.array(v), i))
    landmarks = []
    for sect in sectors:
        chosen = sector[0][0]
        for v, i in sect:
            if np.linalg.norm(v - mean) < np.linalg.norm(chosen - mean):
                chosen = v
        landmarks.append(i)
    return landmarks

def precalculate_landmark_distances(landmarks, adjacency_lists):
    return {i: good_old_dijkstra(adjacency_lists, landmarks[i], None)[1] for i in range(len(landmarks))}

def a_star_with_landmarks(adjacency_lists, node_from, node_to, landmark_distances):
    q = queue.PriorityQueue()
    p = {v: -1 for v in adjacency_lists}
    d = {v: float('inf') for v in adjacency_lists}
    real_d = {}
    d[node_from] = 0
    real_d[node_from] = 0
    q.put((0, node_from))
    while not q.empty():
        dist, cur = q.get()
        if cur == node_to:
            break
        for v in adjacency_lists[cur]:
            edge_len = adjacency_lists[cur][v]
            pi_v = 0
            for L in landmark_distances:
                pi_v = max(res, abs(landmark_distances[L][v] - landmark_distances[L][node_to]))
            if real_d[cur] + edge_len + pi_v < d[v]:
                d[v] = real_d[cur] + edge_len + pi_v
                real_d[v] = real_d[cur] + edge_len
                p[v] = cur
                q.put((d[v], v))
    cur = node_to
    path = []
    while cur != -1:
        path.append(cur)
        cur = p[cur]
    return (reversed(path), real_d[node_to])

In [6]:
import time
from random import randrange

def run_all():
    node_coords = read_node_coords()
    adjacency_lists = read_arcs()
    num_nodes = len(node_coords)
    
    time_start = time.monotonic()
    landmark_distances = precalculate_landmark_distances(choose_landmarks(node_coords), adjacency_lists)
    print('Precalculation done in {:.2} seconds.'.format(time.monotonic() - time_start))
    
    time_dijkstra = 0
    time_a_star = 0
    
    num_tests = 1
    for _ in range(num_tests):
        node_from, node_to = randrange(num_nodes), randrange(num_nodes)
        time_start = time.monotonic()
        good_old_dijkstra(adjacency_lists, node_from, node_to)
        time_dijkstra += time.monotonic()-time_start
        time_start = time.monotonic()
        a_star_with_landmarks(adjacency_lists, node_from, node_to, landmark_distances)
        time_a_star += time.monotonic()-time_start
    print('Time elapsed in {} test: {:.2} second for A* vs. {:.2} seconds for Dijkstra.'.format(num_tests, time_a_star, time_dijkstra))

In [7]:
run_all()

Precalculation done in 2e+02 seconds.
Time elapsed in 1 test: 2.6 second for A* vs. 1.3e+01 seconds for Dijkstra.
